# Definite article tutorial
### Authors: Liz Coppock, Kyle Rawlins

This notebook goes through what is involved in adding a new variable-binding operator to the metalanguage and instantiating it in a lexical entry, by working through iota.

In [1]:
#reload_lamb() # can be uncommented for debugging purposes (this will reload any changes made to the source files)

This notebook walks through the process of adding a new operator, iota, to the lambda notebook.

The notebook has two main parts: the metalanguage, and the object language.  To add iota, we need to modify the meta-language, and then using it in the object language is straightforward.

Every expression in the metalanguage is represented by an object that inherits from the class _TypedExpr_.  Operators that bind a single variable and have a nuclear scope inherit from the subclass _BindingOp_, for example _ExistsUnary_ and _ForallUnary_.  To add Iota, we also subclass from _BindingOp_.  This basically involves specifying type constraints for the variable, the body, and the entire expression.  The default is that the body type and overal type are the same, but this is wrong for iota.  For iota, the body type is type t, and the variable / output type are equal.  A stricter version of this might enforce that both are type e.

To add this to the parser, I also added a hook in the function _TypedExpr.try_parse_op_expr_ in meta.py.  (In the future this should happen more automatically.)

In [2]:
# note: IotaUnary has been added to meta.py, parallel code here for exemplification.
class DemoIotaUnary(meta.BindingOp):
    canonical_name = "Iota" # set the token for parsing this operator
    op_name_uni = "ι"
    op_name_latex = "\\iota{}"
    secondary_names = {"ι"}
    
    def __init__(self, var_or_vtype, body, varname=None, assignment=None, type_check=True):
        super().__init__(var_or_vtype=var_or_vtype, typ=None, varname=varname, body=body, body_type=types.type_t, assignment=assignment, type_check=type_check)
        self.type = self.vartype # output type is the bound variable's type

    def copy(self):
        return DemoIotaUnary(self.vartype, self.body, varname=self.varname)
    
    def copy_local(self, var, arg, type_check=True):
        return DemoIotaUnary(var, arg, type_check=type_check)

meta.BindingOp.add_op(DemoIotaUnary) # add to the registry of binding operators so that it will be parsed.  This will trigger a warning as it overrides the existing IotaUnary class.

WARNING (meta): Overriding existing binding operator 'Iota' in registry


The following examples demonstrate instantiating this class through python class constructions.  (Recall that _lang.te_ calls the metalanguage parser.)

In [3]:
test = DemoIotaUnary("x_e", lang.te("P(x)"))
test

INFO (meta): Coerced guessed type for 'P_t' into <e,t>, to match argument 'x_e'


(ι x_e: P_<e,t>(x_e))

In [4]:
test2 = meta.LFun("f_<e,t>", DemoIotaUnary("x_e", lang.te("f_<e,t>(x_e)")))
test2

(λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))

The following cells demonstrate instatiating iota via the metalanguage parser, and test out combining it with a property.

In [5]:
the = lang.te("L f_<e,t> : Iota x_e : f(x)")
the

(λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))

In [6]:
%%lamb
catexpr = L x_e : Cat(x) # build a property 'catexpr'

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'


${catexpr}_{\left\langle{}e,t\right\rangle{}}\:=\:\lambda{} x_{e} \: . \: {Cat}({x})$

In [7]:
dp = the(catexpr)
dp

((λ f_<e,t>: (ι x_e: f_<e,t>(x_e))))((λ x_e: Cat_<e,t>(x_e)))

In [8]:
dp.reduce()

(ι x_e: ((λ x_e: Cat_<e,t>(x_e)))(x_e))

In [9]:
dp.reduce_all()

(ι x_e: Cat_<e,t>(x_e))

In [10]:
dp.reduce_all().derivation

1. ((λ f_<e,t>: (ι x_e: f_<e,t>(x_e))))((λ x_e: Cat_<e,t>(x_e)))
 2. (ι x_e: ((λ x_e: Cat_<e,t>(x_e)))(x_e))    (Reduction)
 3. (ι x_e: Cat_<e,t>(x_e))    (Recursive reduction of body)

It's worth checking that the type inference is working correctly.  What happens when something not of type t is supplied as the body?  Is the output type right?

The following code catches and displays a TypeMismatch if any.  (To see the full stack trace, you can remove the `try...except` part.)

In [11]:
result = None
try:
    lang.te("Iota x_e : x")
except types.TypeMismatch as e:
    result = e
result

Type mismatch: 'x_e'/e and type t conflict (mode: type adjustment)

In [12]:
lang.te("Iota x_e : P(x)").type

INFO (meta): Coerced guessed type for 'P_t' into <e,t>, to match argument 'x_e'


e

In [13]:
lang.te("Iota x_e : P(x)").__class__

INFO (meta): Coerced guessed type for 'P_t' into <e,t>, to match argument 'x_e'


__main__.DemoIotaUnary

In [14]:
# another way of constructing this:
#test3 = lang.te("P_<e,t>")(lang.te("Iota x_e : Q(x)"))
test3 = lang.te("P_<e,t>(Iota x_e : Q(x))")
test3

INFO (meta): Coerced guessed type for 'Q_t' into <e,t>, to match argument 'x_e'


P_<e,t>((ι x_e: Q_<e,t>(x_e)))

In [15]:
test3.type

t

Now let's use this in a lexical item.  With the metalanguage modifications in place, this is straightforward.

In [16]:
%%lamb
||the|| = L f_<e,t> : Iota x_e : f(x)
||cat|| = L x_e : Cat(x)

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{the}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},e\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}({x})$<br />
$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$

In [17]:
r = the * cat
r.reduce_all()
r

CompositionResult(results=[⟦[the cat]⟧ = (ι x_e: Cat_<e,t>(x_e))], failures=[⟦[cat the]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦the⟧ = (λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))'/<<e,t>,e> conflict (mode: Function Application), ⟦[the cat]⟧ = Type mismatch: '⟦the⟧ = (λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))'/<<e,t>,e> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Modification), ⟦[the cat]⟧ = Type mismatch: '⟦the⟧ = (λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))'/<<e,t>,e> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[cat the]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦the⟧ = (λ f_<e,t>: (ι x_e: f_<e,t>(x_e)))'/<<e,t>,e> conflict (mode: Predicate Abstraction)])

In [18]:
r.tree(derivations=True)

,$[\![\mathbf{\text{[the cat]}}]\!]^{}_{e}$
$=$,"${[\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}({x})]}(\lambda{} x_{e} \: . \: {Cat}({x}))$"
$=$,$\iota{} x_{e} \: . \: {[\lambda{} x_{e} \: . \: {Cat}({x})]}({x})$
$=$,$\iota{} x_{e} \: . \: {Cat}({x})$


What is missing here?  The biggest thing is presuppositions, but interpretation of iota relative to a model would also be helpful.  See the version in lamb.meta for one take on presuppositions.